# Threads and Processes (Reprise)


| metric |**Processes**|**Threads**|
|------|------|------|
|Use all cores|Yes|No
GIL interference|No|Yes
Memory model|Copy on write|Shared
Locking required|Rare|Frequent
Switching|OS, Preemptive|OS, Preemptive
Switching cost|High|High
Size|10s|100s

# So why not use processes all the time?

## Some Answers
1. To ease the difficulty with concurrency 
2. When blocking is involved
3. When you need lots of concurrency

## Let's work an example when blocking is involved

In [ ]:
%%script bash --bg

# This script runs an external process that computes fibonaaci results as a REST resource
python fib.py

In [ ]:
# New and improved fibonacci
import urllib.request

def fibonacci(n):    
    with urllib.request.urlopen(f'http://localhost:5000/fib/{n}') as response:
        html = response.read()
        return int(html)
    
def kill_server():
    with urllib.request.urlopen(f'http://localhost:5000/ex') as response:
        html = response.read()
        return html
    


In [ ]:
%%time
fibonacci(500)

In [ ]:
%%time
import threading

thread1 = threading.Thread(target=fibonacci,args=(36,))
thread1.start()

thread2 = threading.Thread(target=fibonacci,args=(36,))
thread2.start()

thread3 = threading.Thread(target=fibonacci,args=(36,))
thread3.start()

thread4 = threading.Thread(target=fibonacci,args=(36,))
thread4.start()

thread1.join()
thread2.join()
thread3.join()
thread4.join()

In [ ]:
kill_server()

## Cool!  Did we just recover parallelism using Threads?

No, not really.  What we did is made our single thread more efficient.
When each thread invokes a blocking call, the OS will switch to another thread

In our example, each of these two lines has Blocking calls:

```python
with urllib.request.urlopen(f'http://localhost:5000/fib/{n}') as response:
    html = response.read()
                   
```


# One common case where threads are useful if their work involves blocking functions
1. File IO
2. Network IO
3. Sleeping
4. ...


## Footnote: Algorithms and SIMD and GPU really matter.

**Let's take a peak inside that fib.py file**

```python

import numpy
...
def mm_fib(n):
    return (numpy.matrix([[2,1],[1,1]])**(n//2))[0,(n+1)%2]
...
@app.route('/fib/<int:fib_number>')
def do_fib(fib_number):
    time.sleep(1)
    return f"{mm_fib(fib_number)}"
```


In [ ]:
%%script bash --bg

# This script runs an external process that computes fibonaaci results as a REST resource
python fib.py

In [ ]:
%%time
fibonacci(100)